In [1]:
import pandas 
import os
from os import listdir, rename, path
from os.path import isfile, join, exists
from shutil import copyfile

In [2]:
img_path = r'D:\Coding\Python\jupyter\Cards'

In [3]:
data = pandas.read_excel(r'D:\Coding\Python\jupyter\Cards\docs\_CatalogMaster.xlsx', dtype={'ISBN': str})

In [4]:
df = pandas.DataFrame(data, columns=['ISBN', 'ProductCode']).dropna(how='all')

In [5]:
img_files_list = [f for f in listdir(img_path) if isfile(join(img_path, f))]

In [6]:
def try_get_isbn(product_code):
    try:
        line = df.loc[df['ProductCode'] == product_code]
        isbn = (True, line.iloc[0,0])
    except LookupError as e:
        isbn = (False, 'Product code not found')  
    return isbn

In [7]:
with open(join(img_path, "log.txt"), "a") as log_file:
    for image in img_files_list:
        img_product_code = image.split("-")[0]
        img_sub_list = [f for f in listdir(img_path) if img_product_code == f.split("-")[0]]

        for img in img_sub_list:
            isbn = try_get_isbn(img_product_code)
            if isbn[0]:
                if exists(join(img_path, f"{isbn[1]}.jpg")):
                    rename(join(img_path, img), join(img_path, f"{isbn[1]}_1.jpg"))
                    log_file.write(f"{img}, {isbn[1]}_1.jpg\n")
                else:
                    rename(join(img_path, img), join(img_path, f"{isbn[1]}.jpg"))
                    log_file.write(f"{img}, {isbn[1]}.jpg\n")
            else:
                log_file.write(f"{img}, {isbn[1]}\n")
